<a href="https://colab.research.google.com/github/kahochan919/DSAA5024/blob/main/DSAA5024_GroupProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

In [1]:
!pip install plotly pandas

# Setup the folders
!mkdir save_results

FOLDERNAME = '/content/'
import sys
sys.path.append(FOLDERNAME)

# Import libraries
from enum import Enum
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import plotly.express as px
import pandas as pd
import json

import kagglehub
from kagglehub import KaggleDatasetAdapter


mkdir: cannot create directory ‘save_results’: File exists


# Download Data

In [2]:
! gdown --id 1475aFy3_SSYb6JMQ1li7Wx8enO3Nq1k2

! mkdir ~/.kaggle/
! mv kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

# Vaccination progress
! kaggle datasets download gpreda/covid-world-vaccination-progress
# Population Over Countries in 2021
! gdown --id 1nJkmgzjPq6Tzwi4EIUc3oTByDYHrGiaV
# Covid Death Cases from WHO
!wget https://srhdpeuwpubsa.blob.core.windows.net/whdh/COVID/WHO-COVID-19-global-daily-data.csv -O /content/who_death.csv
# Countries Coordinates
# !wget https://raw.githubusercontent.com/plotly/datasets/master/gapminder_with_codes.csv -O countriesCoordinates.csv

! unzip covid-world-vaccination-progress.zip
! rm covid-world-vaccination-progress.zip

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1475aFy3_SSYb6JMQ1li7Wx8enO3Nq1k2
To: /content/kaggle.json
100% 65.0/65.0 [00:00<00:00, 284kB/s]
mkdir: cannot create directory ‘/root/.kaggle/’: File exists
Dataset URL: https://www.kaggle.com/datasets/gpreda/covid-world-vaccination-progress
License(s): CC0-1.0
  0% 0.00/1.94M [00:00<?, ?B/s]
100% 1.94M/1.94M [00:00<00:00, 774MB/s]
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1nJkmgzjPq6Tzwi4EIUc3oTByDYHrGiaV
To: /content/2021_population.csv
100% 17.7k/17.7k [00:00<00:00, 40.7MB/s]
--2025-12-0

In [3]:
# Loading datasets to DataFrame
vaccineDf = pd.read_csv('country_vaccinations.csv')
vaccineManufacturerDf = pd.read_csv('country_vaccinations_by_manufacturer.csv')
populationDf = pd.read_csv('2021_population.csv')
whoDeathDf = pd.read_csv('who_death.csv')

In [4]:
# Check Datasets validity

print('########### Countries Vaccination Records ###########')
print(vaccineDf.head())
print()
print('########### Manufacturer Vaccination Records ###########')
print(vaccineManufacturerDf.head())
print()
print('########### 2021 population ###########')
print(populationDf.head())
print()
print('########### COVID WHO Records ###########')
print(whoDeathDf.head())

########### Countries Vaccination Records ###########
       country iso_code        date  total_vaccinations  people_vaccinated  \
0  Afghanistan      AFG  2021-02-22                 0.0                0.0   
1  Afghanistan      AFG  2021-02-23                 NaN                NaN   
2  Afghanistan      AFG  2021-02-24                 NaN                NaN   
3  Afghanistan      AFG  2021-02-25                 NaN                NaN   
4  Afghanistan      AFG  2021-02-26                 NaN                NaN   

   people_fully_vaccinated  daily_vaccinations_raw  daily_vaccinations  \
0                      NaN                     NaN                 NaN   
1                      NaN                     NaN              1367.0   
2                      NaN                     NaN              1367.0   
3                      NaN                     NaN              1367.0   
4                      NaN                     NaN              1367.0   

   total_vaccinations_per_hundre

# Process Data Frame to valid inputs

## Master Investigation

In [17]:
tempdf = pd.merge(vaccineDf[['country', 'iso_code', 'date', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'daily_vaccinations_raw', 'daily_vaccinations']],
                  populationDf[['iso_code', '2021_last_updated', 'area', 'density_sq_km']], on='iso_code', how='left')
masterdf = pd.merge(tempdf[['country', 'iso_code', 'date', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'daily_vaccinations_raw', 'daily_vaccinations', '2021_last_updated', 'area', 'density_sq_km']],
                  whoDeathDf[['Country', 'New_cases', 'New_deaths', 'Date_reported']],
                  left_on=['country', 'date'],
                  right_on=['Country', 'Date_reported'],
                  how='left')[['country', 'iso_code', 'date', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'daily_vaccinations_raw', 'daily_vaccinations', '2021_last_updated', 'area', 'density_sq_km', 'New_cases', 'New_deaths', 'Date_reported']]

masterdf['2021_last_updated'] = masterdf['2021_last_updated'].str.replace(',', '', regex=False)
masterdf['2021_last_updated'] = pd.to_numeric(masterdf['2021_last_updated'], errors='coerce').astype('Float64')

masterdf['area'] = masterdf['area'].str.replace(' sq_km', '', regex=False)
masterdf['area'] = masterdf['area'].str.replace(',', '', regex=False)
masterdf['area'] = pd.to_numeric(masterdf['area'], errors='coerce').astype('Float64')

masterdf['density_sq_km'] = masterdf['density_sq_km'].str.replace('/sq_km', '', regex=False)
masterdf['density_sq_km'] = masterdf['density_sq_km'].str.replace(',', '', regex=False)
masterdf['density_sq_km'] = pd.to_numeric(masterdf['density_sq_km'], errors='coerce').astype('Float64')

masterdf = masterdf.rename(columns={'2021_last_updated': '2021_population', 'density_sq_km': 'density'})



In [37]:
# Check no populations
# empty_rows = masterdf[masterdf['2021_population'].isnull()]
# countries_list = empty_rows['country'].tolist()

# unique_countries = list(set(countries_list))
# print(unique_countries)

# pop_df = vaccineDf[vaccineDf['country'].isin(['British Virgin Islands', 'England', 'Northern Cyprus', 'Jersey', 'Saint Helena', 'Scotland', 'Wales', 'Pitcairn', 'Bonaire Sint Eustatius and Saba', 'Guernsey', 'Northern Ireland', 'Sint Maarten (Dutch part)', 'Eswatini', 'Kosovo'])]

# unique_iso = pop_df['iso_code'].tolist()
# unique_iso = list(set(unique_iso))
# print(unique_iso)



## Vaccine Investigations

In [33]:
# print(whoDeathDf[whoDeathDf['Country'] == 'Argentina'])

manufacturerDf = vaccineManufacturerDf.drop_duplicates(subset=['location', 'vaccine'], keep='last')
deathDf = whoDeathDf.drop_duplicates(subset=['Country'], keep='last')

tempdf = pd.merge(manufacturerDf[['location', 'vaccine', 'total_vaccinations']],
                  deathDf[['Country', 'Cumulative_cases', 'Cumulative_deaths']],
                  left_on='location', right_on='Country', how='inner')
vaccinedf = tempdf[['location', 'vaccine', 'total_vaccinations', 'Cumulative_cases', 'Cumulative_deaths']]

# print(vaccinedf.head())

    location             vaccine  total_vaccinations  Cumulative_cases  \
0  Argentina             CanSino              610540          10117518   
1  Argentina             Moderna             6507561          10117518   
2  Argentina  Oxford/AstraZeneca            25977231          10117518   
3  Argentina     Pfizer/BioNTech            14681054          10117518   
4  Argentina   Sinopharm/Beijing            28322602          10117518   

   Cumulative_deaths  
0             130820  
1             130820  
2             130820  
3             130820  
4             130820  


# Data Plot

In [35]:
print(masterdf.head())

       country iso_code        date  total_vaccinations  people_vaccinated  \
0  Afghanistan      AFG  2021-02-22                 0.0                0.0   
1  Afghanistan      AFG  2021-02-23                 NaN                NaN   
2  Afghanistan      AFG  2021-02-24                 NaN                NaN   
3  Afghanistan      AFG  2021-02-25                 NaN                NaN   
4  Afghanistan      AFG  2021-02-26                 NaN                NaN   

   people_fully_vaccinated  daily_vaccinations_raw  daily_vaccinations  \
0                      NaN                     NaN                 NaN   
1                      NaN                     NaN              1367.0   
2                      NaN                     NaN              1367.0   
3                      NaN                     NaN              1367.0   
4                      NaN                     NaN              1367.0   

   2021_population      area  density  New_cases  New_deaths Date_reported  
0       3

In [38]:
print(vaccinedf.head())

    location             vaccine  total_vaccinations  Cumulative_cases  \
0  Argentina             CanSino              610540          10117518   
1  Argentina             Moderna             6507561          10117518   
2  Argentina  Oxford/AstraZeneca            25977231          10117518   
3  Argentina     Pfizer/BioNTech            14681054          10117518   
4  Argentina   Sinopharm/Beijing            28322602          10117518   

   Cumulative_deaths  
0             130820  
1             130820  
2             130820  
3             130820  
4             130820  
